<a href="https://colab.research.google.com/github/vacodoceo/vacodoceo-IIC3633-2020/blob/master/practicos/pyRecLab_SlopeOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Verner Codoceo

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?

- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?

- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)

- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?

- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

1. Porque se basa en las calificaciones de otros usuarios para generar recomendaciones
2. En el los *items*. Esto porque *Slope One* se basa en los *items* que el usuario a recomendar haya calificado anteriormente, con el fin de buscar "patrones" (la desviación) en *items* y así dar con una predicción dadas las desviaciones de los *items* que ya ha calificado el usuario a recomendar.
3. Recibe una matriz de usuarios x items donde las entradas son las calificaciones entregadas. *Slope One* calula las desviaciones de calificación entre un *item* a saber su recomendación y los demás items. Esta desviación puede verse como la diferencia promedio entre las calificaciones de distintos usuarios. Luego, se toma calcula una predicción de calificación sumandole a cada calificación del usuario su desviación hacia el *item* a predecir correspondiente, y luego se calcula la suma ponderada de estas calificaciones + desviación.
4. Habría que actualizar las desviaciones desde y hacia el *item* que se calificó.
5. Mal, ya que habría pocas desviaciones utilizables, por lo que se estaría prediciendo en base a una muestra pequeña de datos.




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    380      0 --:--:--  0:00:01 --:--:--   380
100 1546k  100 1546k    0     0   895k      0  0:00:01  0:00:01 --:--:--  895k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    734      0 --:--:-- --:--:-- --:--:--   733
100  385k  100  385k    0     0   432k      0 --:--:-- --:--:-- --:--:--  432k


In [4]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1124      0 --:--:-- --:--:-- --:--:--  1124
100  230k  100  230k    0     0   356k      0 --:--:-- --:--:-- --:--:--  356k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [6]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 235kB 2.7MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [7]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [8]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [9]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [10]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:** Calcula las desviaciones y número de usuarios que se utilizó para cada desviación para todos los pares de *items*.

## Paso 6:

Llego la hora de predecir el rating.

In [11]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [12]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [13]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [14]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Esto se explica debido a que el número de desviaciones hacia el *item* 9 puede y es probable que sea diferente al del *item* 1168. De la misma manera, el número de usuarios utilizados por desviación también es probable que sea diferente.

In [15]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [17]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [18]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Sí. Esto se debe a que probablemente los *items* que el usuario 457 calificó sean distintos a los que calificó el usuario 1057, por lo que se utilizarán distintas desviaciones para generar las recomendaciones. Por ej, si el usuario 457 solo ha calificado las películas 1, 2, 3, solo se utilizarán las desviaciones proveniente de esas 3 películas, mientras que si el usuario solo calificó la 4, 5, 6, solo se utilizarán las desviaciones provenientes de esas otras 3 películas.

In [20]:
reclist_slopeone = mySlopeOne.recommend("1057", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
600         Daniel Defoe's Robinson Crusoe (1996)
1682    Scream of Stone (Schrei aus Stein) (1991)
1681                          You So Crazy (1994)
1080                   Celestial Clockwork (1994)
1422                                Suture (1993)
1680                         Sliding Doors (1998)
1679                             B. Monkey (1998)
1678                            Mat' i syn (1997)
1677                         Sweet Nothing (1995)
1676                      War at Home, The (1996)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** Se puede notar que las recomendaciones no tienen ninguna intersección. Esto no es de extrañar, ya que ambas se basan en información completamente diferente. 

A modo de ejemplo, si muchos usuarios que califican muy mal califican medianamente bien a un *item*, este *item* tendrá muy "buenas" desviaciones y lo más probable es que aparecerá en las recomendaciones de *Slope One*. Por otro lado, probablemente **no** aparecerá en *Most Popular*, ya que este preferirá *items* con altas calificaciones en lugar de los con buenas calificaciones "relativas".

In [28]:
# Slope One
reclist_slopeone = mySlopeOne.recommend("44", 10)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre para Slope One:')
info_file.loc[recmovies_slopeone]['title']




Lista de items por nombre para Slope One:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
Name: title, dtype: object

In [27]:
# Most Popular
most_popular = pyreclab.MostPopular(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

most_popular.train()

top_n = 10
user_id = 44

ranking_most_popular = [int(r) for r in most_popular.recommend(str(user_id), top_n, includeRated=False)]
info_file.loc[ranking_most_popular]['title']

movieid
50                     Star Wars (1977)
100                        Fargo (1996)
181           Return of the Jedi (1983)
286         English Patient, The (1996)
1                      Toy Story (1995)
121       Independence Day (ID4) (1996)
300                Air Force One (1997)
127               Godfather, The (1972)
7                 Twelve Monkeys (1995)
98     Silence of the Lambs, The (1991)
Name: title, dtype: object